In [1]:
import pexpect, subprocess
import time
from datetime import datetime
from pexpect_handler import PexpectHandler
from logger import setup_logger

# Setup logger and shell handler (used if needed)
logger = setup_logger()
handler = PexpectHandler(command="bash")
handler.spawn_process()


def read_file(filepath):
    """
    Reads the contents of a file and returns it as a string.

    Args:
        filepath (str): Path to the file.

    Returns:
        str: File contents.

    Raises:
        FileNotFoundError: If the file doesn't exist.
        IOError: If there is an error reading the file.
    """
    with open(filepath, "r", encoding="utf-8") as file:
        return file.read()


def run_shell_command(cmd):
    """
    Runs a shell command and returns its output as a string.

    Args:
        cmd (str): The shell command to run.

    Returns:
        str: The standard output from the command.

    Raises:
        subprocess.CalledProcessError: If the command exits with a non-zero status.
    """
    result = subprocess.run(
        cmd,
        shell=True,
        check=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
    )
    return result.stdout.strip()


def send_promptbased_command(
    regex_pattern,
    input_file,
    command_to_send,
    tmux_session,
    windows_name,
    pane_index,
    last_n_lines=30,
):
    # Execute shell command tail -n <last_n_lines> <input_file>
    child = pexpect.spawn(f"tail -n {last_n_lines} {input_file}")
    index = child.expect([regex_pattern, pexpect.TIMEOUT, pexpect.EOF], timeout=10)

    if index == 0:
        print(
            f"✅ Pattern '{regex_pattern}' found in the last {last_n_lines} lines of {input_file}."
        )
        tmux_command = f"tmux send-keys -t {tmux_session}:{windows_name}.{pane_index} '{command_to_send}' C-m"
        pexpect.run(tmux_command)
        print(f"➡️ Sent command to tmux pane: {tmux_command}")
        child.close()
        return 1
    elif index == 1:
        print(f"⏱️ Timeout while waiting for pattern '{regex_pattern}'.")
    elif index == 2:
        print(f"📄 End of file reached without finding pattern '{regex_pattern}'.")

    child.close()
    return 0  # treat both timeout and EOF as failure


def get_local_host_time():
    return datetime.now().strftime("+%m%d%H%M%Y.%S")


def run_with_retries(
    pattern,
    command,
    lines,
    input_file,
    tmux_session,
    window_name,
    pane_index,
    max_retries=3,
):
    attempt = 0
    while attempt < max_retries:
        result = send_promptbased_command(
            pattern,
            input_file,
            command,
            tmux_session,
            window_name,
            pane_index,
            last_n_lines=lines,
        )
        if result == 1:
            return True  # success
        attempt += 1
        if attempt == 1:
            print(f"🔁 [Retry {attempt}] Waiting 15 seconds before retrying...")
            time.sleep(15)
        else:
            print(f"🔁 [Retry {attempt}] Waiting 30 seconds before retrying...")
            time.sleep(30)

    # All retries failed
    raise RuntimeError(
        f"❌ Command failed after {max_retries} attempts: pattern='{pattern}', command='{command}'"
    )


def intial_login(steps, input_file, tmux_session, windows_name, pane_index):
    for pattern, command, lines in steps:
        try:
            run_with_retries(
                pattern,
                command,
                lines,
                input_file,
                tmux_session,
                windows_name,
                pane_index,
            )
            time.sleep(2)
        except RuntimeError as e:
            print(e)
            break


2025-10-02 19:09:42,340 - INFO - Spawned process: bash


In [3]:
# List of (regex pattern, command to send, number of log lines to check)
steps = [
    ("MyDevelopment\\$ ", "telnet 172.25.27.3 2016", 5),
    ("\\]\\'.", "", 5),
    ("login:", "admin", 5),
    ("Password:", "Nbv12345", 5),
]

steps2 = [
    ("login:", "admin", 5),
    ("Password:", "Nbv12345", 5),
]

# Execute steps
tmux_session = "my_session"
windows_name = "GE"
pane_index = 9
blade_index = 6 
input_file = f"../logs/BMC{blade_index}.log"

login = [
    ("login:", "root", 5),
    ("Password:", "Free4All", 5),
]

cpu_mon=[
    (r"\# ", "cd /home/hai/stress_tools && resize", 5),
    (r"\# ", f"screen -dmS mon script -f B{blade_index}/mon{blade_index}.log", 5),
    (r"\# ", f"screen -dmS ipmi script -f B{blade_index}/ipmi{blade_index}.log", 5),
    (r"\# ", f"screen -dmS nvqual script -f B{blade_index}/nvqual{blade_index}.log", 5),
    (r"\# ", f"screen -dmS traffic script -f B{blade_index}/traffic{blade_index}.log", 5),
    (r"\# ", f"screen -dmS stress script -f B{blade_index}/stress{blade_index}.log", 5),
    (r"\# ", f"screen -dmS fio script -f B{blade_index}/fio{blade_index}.log", 5),
    (r"\# ", "screen -rd mon", 5),
    (r"\# ", "/home/hai/stress_tools/start.sh && sleep 5 && echo hello", 5),
    (r"\# ", " ", 5),
    (r"\# ", "tail -F *ptatmon.csv", 5),
]

ipmi = [
    (r"\# ", "screen -rd ipmi", 5),
    (r"\# ", "cd /home/hai/stress_tools", 5),
    (r"\# ", "sh /home/hai/stress_tools/record_ipmi.sh", 5),
]



In [5]:
intial_login(login, input_file, tmux_session, windows_name, pane_index)
date=f"{get_local_host_time()}"
set_date = f"tmux send-keys -t {tmux_session}:{windows_name}.{pane_index} 'export TZ=UTC && date {date}' C-m"
run_shell_command(set_date)
print(f"➡️ Sent command to tmux pane: {set_date}")

intial_login(cpu_mon, input_file, tmux_session, windows_name, pane_index)
detach_screen = f"tmux send-keys -t {tmux_session}:{windows_name}.{pane_index} C-a d"
run_shell_command(detach_screen)
print(f"➡️ Sent command to tmux pane: {detach_screen}")

intial_login(ipmi, input_file, tmux_session, windows_name, pane_index)
detach_screen = f"tmux send-keys -t {tmux_session}:{windows_name}.{pane_index} C-a d"
run_shell_command(detach_screen)
print(f"➡️ Sent command to tmux pane: {detach_screen}")

✅ Pattern 'login:' found in the last 5 lines of ../logs/BMC6.log.
➡️ Sent command to tmux pane: tmux send-keys -t my_session:GE.9 'root' C-m
✅ Pattern 'Password:' found in the last 5 lines of ../logs/BMC6.log.
➡️ Sent command to tmux pane: tmux send-keys -t my_session:GE.9 'Free4All' C-m
➡️ Sent command to tmux pane: tmux send-keys -t my_session:GE.9 'export TZ=UTC && date +100219122025.36' C-m
✅ Pattern '\# ' found in the last 5 lines of ../logs/BMC6.log.
➡️ Sent command to tmux pane: tmux send-keys -t my_session:GE.9 'cd /home/hai/stress_tools && resize' C-m
✅ Pattern '\# ' found in the last 5 lines of ../logs/BMC6.log.
➡️ Sent command to tmux pane: tmux send-keys -t my_session:GE.9 'screen -dmS mon script -f B6/mon6.log' C-m
✅ Pattern '\# ' found in the last 5 lines of ../logs/BMC6.log.
➡️ Sent command to tmux pane: tmux send-keys -t my_session:GE.9 'screen -dmS ipmi script -f B6/ipmi6.log' C-m
✅ Pattern '\# ' found in the last 5 lines of ../logs/BMC6.log.
➡️ Sent command to tmux pa

In [6]:
remove_old_key = f"tmux send-keys -t {tmux_session}:{windows_name}.{pane_index} C-a d"
run_shell_command(remove_old_key)
print(f"➡️ Sent command to tmux pane: {remove_old_key}")


➡️ Sent command to tmux pane: tmux send-keys -t my_session:GE.6 C-a d
